In [1]:
# Load the dataset form firestore
# Setup with GPU
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%load_ext autoreload
%autoreload 2
import tensorflow as tf
tf.config.list_physical_devices('GPU')

import json
from usl_models.flood_ml.trainingdataset import IncrementalTrainDataGenerator

sim_names = ["Manhattan-config_v1%2FRainfall_Data_1.txt"]

generator = IncrementalTrainDataGenerator()
dataset = generator.load_dataset_windowed(sim_names, batch_size=0, max_chunks=1)


env: TF_FORCE_GPU_ALLOW_GROWTH=true


2024-07-12 22:43:52.991591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 22:43:52.991637: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 22:43:52.993044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 22:43:53.001284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 22:43:55.205858: I external/local_xla/xla/

In [3]:
inputs, labels = next(iter(dataset))

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_1_3.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/1_3.npy
Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


In [4]:
@tf.function
def serialize_example(inputs: dict):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  print(inputs['spatiotemporal'].shape)BytesList
  # Create a Features message using tf.train.Example.
  example = tf.train.Example(features = tf.train.Features(
    feature={
      k: tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(v)]))
      for k, v in inputs.items()
    }
  ))
  return example.SerializeToString()
serialize_example(inputs)

(5, 1000, 1000, 1)


TypeError: in user code:

    File "/var/tmp/ipykernel_49497/1827984150.py", line 8, in serialize_example  *
        example = tf.train.Example(features = tf.train.Features(

    TypeError: <tf.Tensor 'SerializeTensor:0' shape=() dtype=string> has type SymbolicTensor, but expected one of: bytes


In [20]:

writer = tf.data.experimental.TFRecordWriter("data/batch_pred.tfrecord")
writer.write(inputs)

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_2_5.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/2_5.npy
Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


TypeError: Invalid `dataset.` Expected a `tf.data.Dataset` object but got <class 'dict'>.

In [3]:
inputs, labels = next(iter(dataset))
inputs = { k: v.numpy().tolist() for k, v in inputs.items() }
outfile = 'data/batch_pred_josiahkp.jsonl'

# Writing the data to the JSONL file
with open(outfile, 'w') as f:
    json.dump(inputs, f)


Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_4_7.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/4_7.npy
Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)
